# Geospatial Python: Representing and Processing Vector Geometries

This notebook is part of a geospatial Python series that introduces how to work with vector geometries using libraries like Shapely, Fiona, and PyProj. 

## Overview
We will cover:
- Creating and analyzing vector geometries (points, lines, and polygons)
- Performing spatial operations (buffer, intersection, and union)
- Reading and writing shapefiles using Fiona
- Performing coordinate transformations with PyProj

## 1. Representing Vector Geometries: Points, Lines, and Polygons

In [ ]:
import os
from shapely.geometry import Point, LineString, Polygon

### Points
A `Point` represents a single coordinate in space. 
The following example creates two points and calculates the distance between them.

In [ ]:
point = Point(1, 1, 2)  # A 3D point (X, Y, Z)
point_2 = Point(2, 2)  # A 2D point (X, Y)

# Compute the Euclidean distance between the two points
# Note: The distance is computed in 2D (ignoring Z)
distance = point.distance(point_2)
distance

### Lines
A `LineString` represents a sequence of points connected by straight-line segments.
Below, we create a simple line and compute its length.

In [ ]:
line = LineString([(1, 1), (2, 3)])
print("Line length:", line.length)

### Polygons
A `Polygon` is defined by a sequence of coordinates forming a closed shape.
We define a polygon and compute its perimeter.

In [ ]:
polygon = Polygon([(30, 10), (40, 40), (20, 40), (10, 20), (30, 10)])
print("Polygon perimeter:", polygon.length)

## 2. Spatial Operations
We will perform various spatial operations such as buffering, intersection, and union.

In [ ]:
new_point = Point(4, 4)
new_line = LineString([(1, 1), (3, 3)])

# Creating a buffer around the point
buffer = new_point.buffer(distance=5)  # Creates a circular buffer

# Creating a single-sided buffer around the line (when single_sided is True, a negative distance puts the buffer on the opposite side)
line_buffer = new_line.buffer(distance=-3, single_sided=True)

# Checking if a point is within a buffered region
new_point_2 = Point(6, 1)
is_within = new_point_2.within(buffer)
print("Point within buffer:", is_within)

# Computing intersection of buffer and line buffer
intersection = buffer.intersection(line_buffer)
# print(intersection)

# Computing union of two geometries
union = line_buffer.union(new_line)
print("Union result:", union)

## 3. Reading and Writing Shapefiles with Fiona
Fiona allows us to read and write geospatial data in shapefile format.

In [ ]:
import fiona

# Define directory to store shapefiles
dir = os.path.join(os.getcwd(), "shp")
os.makedirs(dir, exist_ok=True)

shapefile_path = os.path.join(dir, 'file.shp')

# Define the schema for the shapefile
schema = {
    'geometry': 'Point',
    'properties': {
        'name': 'str',
        'desc': 'str',
    }
}

# Create a list of point features
file_points = [
    {'coordinates': (0, 0), 'name': 'Point 1', 'desc': 'This is point 1'},
    {'coordinates': (1, 0), 'name': 'Point 2', 'desc': 'This is point 2'},
    {'coordinates': (0, 1), 'name': 'Point 3', 'desc': 'This is point 3'},
    {'coordinates': (1, 1), 'name': 'Point 4', 'desc': 'This is point 4'}
]

# Writing points to a shapefile
with fiona.open(
    shapefile_path, 'w', 
    schema=schema, 
    driver='ESRI Shapefile',
    crs='EPSG:4326'
    ) as file:
    for item in file_points:
        point = {
            'geometry': {'type': 'Point', 'coordinates': item['coordinates']},
            'properties': {'name': item['name'], 'desc': item['desc']}
        }
        file.write(point)

# Reading from a shapefile
with fiona.open(shapefile_path, 'r') as dst:
    for feature in dst:
        print("Feature:", feature)

## 4. Coordinate Transformations with PyProj
PyProj is used for transforming coordinates between different coordinate reference systems (CRS).

In [ ]:
from pyproj import Transformer, CRS

# Define a transformer from WGS 84 (EPSG:4326) to UTM Zone 30N (EPSG:32630)
transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32630', always_xy=True)

# Convert coordinates from longitude/latitude to UTM
x, y = transformer.transform(-1.570523, 6.684967)
print("Transformed coordinates:", x, y)

# Retrieve and display CRS information
crs = CRS.from_epsg(4326)
print("CRS Info:", crs)